In [ ]:
import numpy as np
import joblib  # for saving the model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA, PCA
from sklearn.neighbors import KNeighborsClassifier
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy' % (n))
y = np.load('Datasets/kryptonite-%s-y.npy' % (n))

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # 10% of all data for final model evaluation

# To hold the best model and accuracy score
best_model = None
best_accuracy = -1

def objective(trial):
    global best_model, best_scaler, best_accuracy, best_pca

    # Shuffle and split data
    X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.8, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    pca = PCA(n_components=trial.suggest_int('n_components', 1, X_train_scaled.shape[1]))
    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_val_scaled = pca.transform(X_val_scaled)

    # KNN Classifier parameters
    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()

    # If this is the best model so far, save it
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = classifier  # Update the best model
        best_scaler = scaler
        best_pca = pca
        # Save the best model
        joblib.dump(best_model, 'best_knn_model.joblib')
        print(f"New best model saved with accuracy: {best_accuracy:.4f}")

    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner(n_startup_trials=10)
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)
print("Best model saved as 'best_knn_model.joblib'.")

# Optional: Visualize optimization results
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

# Evaluate on the validation set
X_test_scaled = best_scaler.transform(X_test)
X_test_scaled = best_pca.transform(X_test_scaled)
y_test_pred = best_model.predict(X_test_scaled)
val_accuracy = accuracy_score(y_test, y_test_pred)


[I 2024-11-12 19:10:06,658] A new study created in memory with name: no-name-587c3cf7-90e9-4094-9f40-1788a8ccbd50
[I 2024-11-12 19:10:07,180] Trial 0 finished with value: 0.6910493827160494 and parameters: {'n_components': 3, 'n_neighbors': 15, 'weights': 'uniform'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:07,220] Trial 1 finished with value: 0.5084104938271605 and parameters: {'n_components': 1, 'n_neighbors': 31, 'weights': 'distance'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:07,268] Trial 2 finished with value: 0.508641975308642 and parameters: {'n_components': 1, 'n_neighbors': 40, 'weights': 'distance'}. Best is trial 0 with value: 0.6910493827160494.


New best model saved with accuracy: 0.6910


[I 2024-11-12 19:10:07,910] Trial 3 finished with value: 0.5515432098765433 and parameters: {'n_components': 6, 'n_neighbors': 28, 'weights': 'uniform'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:07,959] Trial 4 finished with value: 0.55625 and parameters: {'n_components': 2, 'n_neighbors': 17, 'weights': 'distance'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:08,475] Trial 5 finished with value: 0.6345679012345679 and parameters: {'n_components': 3, 'n_neighbors': 41, 'weights': 'uniform'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:09,139] Trial 6 finished with value: 0.4613425925925926 and parameters: {'n_components': 7, 'n_neighbors': 18, 'weights': 'uniform'}. Best is trial 0 with value: 0.6910493827160494.
[I 2024-11-12 19:10:09,581] Trial 7 finished with value: 0.5511574074074074 and parameters: {'n_components': 2, 'n_neighbors': 8, 'weights': 'uniform'}. Best is trial 0 with value: 0.6910493827160494.
[

New best model saved with accuracy: 0.7586
New best model saved with accuracy: 0.9313


[I 2024-11-12 19:10:09,947] Trial 11 finished with value: 0.9232253086419753 and parameters: {'n_components': 9, 'n_neighbors': 1, 'weights': 'distance'}. Best is trial 9 with value: 0.9313271604938271.
[I 2024-11-12 19:10:10,011] Trial 12 finished with value: 0.9232253086419753 and parameters: {'n_components': 9, 'n_neighbors': 1, 'weights': 'distance'}. Best is trial 9 with value: 0.9313271604938271.
[I 2024-11-12 19:10:10,192] Trial 13 finished with value: 0.9155092592592593 and parameters: {'n_components': 7, 'n_neighbors': 9, 'weights': 'distance'}. Best is trial 9 with value: 0.9313271604938271.
[I 2024-11-12 19:10:10,386] Trial 14 finished with value: 0.9202160493827161 and parameters: {'n_components': 8, 'n_neighbors': 8, 'weights': 'distance'}. Best is trial 9 with value: 0.9313271604938271.
[I 2024-11-12 19:10:10,606] Trial 15 finished with value: 0.7787808641975309 and parameters: {'n_components': 5, 'n_neighbors': 49, 'weights': 'distance'}. Best is trial 9 with value: 0.93

...

[I 2024-11-12 19:10:11,894] Trial 23 finished with value: 0.8921296296296296 and parameters: {'n_components': 7, 'n_neighbors': 13, 'weights': 'distance'}. Best is trial 21 with value: 0.937037037037037.
[I 2024-11-12 19:10:11,986] Trial 24 pruned. 
[I 2024-11-12 19:10:12,199] Trial 25 finished with value: 0.8963734567901235 and parameters: {'n_components': 7, 'n_neighbors': 12, 'weights': 'distance'}. Best is trial 21 with value: 0.937037037037037.
[I 2024-11-12 19:10:12,717] Trial 26 finished with value: 0.8959876543209877 and parameters: {'n_components': 6, 'n_neighbors': 6, 'weights': 'uniform'}. Best is trial 21 with value: 0.937037037037037.
[I 2024-11-12 19:10:13,042] Trial 27 pruned. 
[I 2024-11-12 19:10:13,143] Trial 28 pruned. 
[I 2024-11-12 19:10:13,814] Trial 29 pruned. 
[I 2024-11-12 19:10:13,886] Trial 30 finished with value: 0.9010802469135802 and parameters: {'n_components': 5, 'n_neighbors': 4, 'weights': 'distance'}. Best is trial 21 with value: 0.937037037037037.
[I 

New best model saved with accuracy: 0.9394


[I 2024-11-12 19:10:16,024] Trial 39 finished with value: 0.9113425925925925 and parameters: {'n_components': 7, 'n_neighbors': 2, 'weights': 'uniform'}. Best is trial 38 with value: 0.939429012345679.
[I 2024-11-12 19:10:16,293] Trial 40 pruned. 
[I 2024-11-12 19:10:16,372] Trial 41 finished with value: 0.9342592592592592 and parameters: {'n_components': 6, 'n_neighbors': 4, 'weights': 'distance'}. Best is trial 38 with value: 0.939429012345679.
[I 2024-11-12 19:10:16,493] Trial 42 finished with value: 0.9277006172839506 and parameters: {'n_components': 7, 'n_neighbors': 7, 'weights': 'distance'}. Best is trial 38 with value: 0.939429012345679.
[I 2024-11-12 19:10:16,562] Trial 43 pruned. 
[I 2024-11-12 19:10:16,736] Trial 44 finished with value: 0.9215277777777777 and parameters: {'n_components': 7, 'n_neighbors': 8, 'weights': 'distance'}. Best is trial 38 with value: 0.939429012345679.
[I 2024-11-12 19:10:16,785] Trial 45 finished with value: 0.9195216049382716 and parameters: {'n_

Best hyperparameters: {'n_components': 7, 'n_neighbors': 3, 'weights': 'distance'}
Best value: 0.939429012345679
Best model saved as 'best_knn_model.joblib'.


ValueError: X has 9 features, but KNeighborsClassifier is expecting 7 features as input.